* Importação de bibliotecas e carregando vídeo

In [ ]:

import moviepy.editor as mpe
import numpy as np 
from moviepy.audio.fx.all import audio_fadeout
video = mpe.VideoFileClip("2025-06-05-20-16-53.mp4")



* Inverter a imagem a cada 20 segundos Dividir o vídeo em blocos de 20 segundos.

Utilizar uma função do numpy que  inverte as entradas em cada linha na direção esquerda/direita. Depois uso fl() (uma funcao do MoviePy) para modificar frames usando a função fliF_Frame que criei.

In [11]:
def flip_frame(get_frame, t):
    segment = int(t // 20)
    frame = get_frame(t)
    if segment % 2 == 1:
        frame = np.fliplr(frame)
    return frame

flipped_video = video.fl(lambda gf, t: flip_frame(gf, t))



* Reduzir volume gradualmente a cada 30s e deixar os últimos 10s em silêncio

A função calculate_fade(t, total_duration, fade_duration) gera um fator de volume que é 1 até o tempo fade_start = total_duration - fade_duration.

epois desse ponto (fade_zone), o volume decresce linearmente até zero no final do clipe.

Na função volume_adjustment, para cada frame de áudio em tempo t, aplica-se esse fator para multiplicar a amplitude do áudio, fazendo o volume ir caindo até o silêncio.

In [ ]:
def calculate_fade(t, total_duration, fade_duration=10):
    fade_start = total_duration - fade_duration
    fade_factor = np.ones_like(t, dtype=float)
    
    fade_zone = t >= fade_start
    linear_fade = 1 - (t[fade_zone] - fade_start) / fade_duration
    
    fade_factor[fade_zone] = linear_fade ** 2 
    
    fade_factor = np.clip(fade_factor, 0, 1)
    return fade_factor

def add_gradual_fadeout(clip):
    if clip.audio is None:
        print("O clipe não possui áudio.")
        return clip
    
    def volume_adjustment(get_frame, times):
        audio_frame = get_frame(times)
        times = np.array(times, ndmin=1)
        
        volume = calculate_fade(times, clip.duration)
        
        if audio_frame.ndim == 2:
            volume = volume[:, np.newaxis]
        
        return audio_frame * volume

    faded_audio = clip.audio.fl(volume_adjustment)
    return clip.set_audio(faded_audio)

video_fadeout = add_gradual_fadeout(flipped_video)

* Após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80  
Por fim, insira os 20 segundos cortados acima no fim do video 

Usei o método .subclip(inicio, fim) para extrair trechos específicos do vídeo.

Usa concatenate_videoclips para juntar esses trechos na ordem desejada.



In [13]:
start_cut = 60 
end_cut = 80   


clip_start = video_fadeout.subclip(0, start_cut)

clip_cut = video_fadeout.subclip(start_cut, end_cut)

clip_end = video_fadeout.subclip(end_cut, video_fadeout.duration)

video_sem_corte = mpe.concatenate_videoclips([clip_start, clip_end])

video_final = mpe.concatenate_videoclips([video_sem_corte, clip_cut])

video_final.write_videofile("video_editado_final.mp4")

Moviepy - Building video video_editado_final.mp4.
MoviePy - Writing audio in video_editado_finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_editado_final.mp4



Moviepy - Done !
Moviepy - video ready video_editado_final.mp4
